# 本筆記的目的：預測重症病患未來的生還率。

[PhysioNet 2012](https://physionet.org/challenge/2012/) 資料集紀錄了各個病患，於離開加護病房後是否有生還。

並且，這些病人，從進入加護病房，至離開，會有各式各樣的生理資訊，於各個時間點被記錄下來。

我們的目的是以這個資料集來建立模型，去讓機器預測一個重症病患進入加護病房後，未來可能會有多少的生還率。

---

# 索引
* [1. 載入 & 整理資料](#01)
    * [整理出存放特徵欄位的Tensor，分別為train_x, val_x和test_x。](#011)
    * [整理出存放標籤(生存/死亡)的Tensor，分別為train_y, val_y和test_y。](#012)
    * [儲存待訓練資料至hdf5資料庫](#013)
    * [由hdf5資料庫載入待訓練資料](#014)
    * [檢查資料是否有空值](#015)
    * [資料標準化](#016)
    * [將空值做填補](#017)
    * [將所有病人的時序資料截斷或補零至相同長度](#018)
* [2. 建立模型](#02)
* [3. 訓練模型](#03)

---

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
from IPython.display import display
import re

import seaborn as sns
sns.set()

## <a id=01>1. 載入 & 整理資料</a>

看某個病人的數據

In [ ]:
# !cat ../datasets/physioNet-set-a/134976.txt

看有幾個病人

In [ ]:
!ls -hl ../datasets/physioNet-set-a | wc -l

In [ ]:
def icu_df_creator(path,verbose=True):
    '''由txt檔載入病人資訊，存成資料表。'''
    df=pd.read_csv(path)
    if verbose:
        print("Before pivot transform:")
        display( df.head(5) )
        print(df.shape)

    encounter_id=re.findall(".*/(.*).txt",path)[0]

    df["encounter_ID"]=int(encounter_id)
    df=df.pivot_table(index=["encounter_ID","Time"],
                      columns="Parameter",
                      values="Value")
    df.columns.name=""
    df = df.astype(np.float32)
    df.drop(columns="RecordID",inplace=True)
    if verbose:
        print("\nAfter pivot transform:")
        display( df.head(5) )
        print(df.shape)
    return df

---

In [ ]:
def filePathsGen(rootPath):
    '''此函數將rootPath資料夾目錄中的所有檔案路徑資訊儲存至一個清單內。'''
    paths=[]
    dirs=[]
    for dirPath,dirNames,fileNames in os.walk(rootPath):
        for fileName in fileNames:
            fullPath=os.path.join(dirPath,fileName)
            paths.append( fullPath) 
        dirs.append(dirNames)
    return dirs,paths

In [ ]:
root_folder="../datasets/physioNet-set-a"
_,paths = filePathsGen(root_folder)

dfs=[]
for idx,path in enumerate(paths):
    if idx%200==0:
        print(idx,path)
    if "Outcomes-a" in path:
        continue
    dfs.append( icu_df_creator(path,verbose=False) 
              )

將病人資訊資料表整理出來:

In [ ]:
df = pd.concat(dfs, sort=False)

檢視病人資訊資料表有哪些欄位:

In [ ]:
df.info()

In [ ]:
df.head(5) #取該資料表的前五筆出來檢視

#### 練習: 選出某個病患的呼吸率(RespRate)時序，並將其繪製出來。

In [ ]:
df.loc[138633]["RespRate"].plot(marker="o",lw=1)

[[Back to Top]](#索引)

### <a id=011>整理出存放特徵欄位的Tensor，分別為train_x, val_x和test_x。</a>

In [ ]:
# 資料切成 70% train, 15% validation, 15% test

pids = pd.Series( df.index.levels[0] )
pids_train_x = pids.sample(frac=0.7)
pids_test_x = pids.drop(index=pids_train_x.index)
pids_val_x = pids_test_x.sample(frac=0.5)
pids_test_x = pids_test_x.drop(index=pids_val_x.index)

print(pids_train_x.shape)
print(pids_val_x.shape)
print(pids_test_x.shape)

In [ ]:
train_x = df.loc[pids_train_x.values]
val_x = df.loc[pids_val_x.values]
test_x = df.loc[pids_test_x.values]

---

[[Back to Top]](#索引)

### <a id=012>整理出存放標籤(生存/死亡)的Tensor，分別為train_y, val_y和test_y。</a>

In [ ]:
# !cat ../datasets/physioNet-set-a/Outcomes-a.txt # 欄位：In-hospital_death表示
                                                  # 該病患是否最終有生還。我們要以
                                                  # 該欄位作為我們的預測標的。

In [ ]:
icu_data_y = pd.read_csv("../datasets/physioNet-set-a/Outcomes-a.txt", index_col="RecordID")
icu_data_y = icu_data_y["In-hospital_death"]

In [ ]:
# 整理出train_y, val_y和test_y

pids_train = train_x.reset_index()["encounter_ID"].unique()
pids_val = pd.Series(val_x.reset_index()["encounter_ID"].unique())
pids_test = pd.Series(test_x.reset_index()["encounter_ID"].unique())

train_y = icu_data_y.loc[pids_train]
val_y = icu_data_y.loc[pids_val]
test_y = icu_data_y.loc[pids_test]

---

[[Back to Top]](#索引)

### <a id=013>儲存待訓練資料至hdf5資料庫</a>

In [ ]:
train_x.to_hdf('ICU_PhysioNet2012.h5', key='train_x', mode='w')
train_y.to_hdf('ICU_PhysioNet2012.h5', key='train_y', mode='a')

val_x.to_hdf('ICU_PhysioNet2012.h5', key='val_x', mode='a')
val_y.to_hdf('ICU_PhysioNet2012.h5', key='val_y', mode='a')

test_x.to_hdf('ICU_PhysioNet2012.h5', key='test_x', mode='a')
test_y.to_hdf('ICU_PhysioNet2012.h5', key='test_y', mode='a')

In [ ]:
!ls -hl ICU_PhysioNet2012.h5

---

[[Back to Top]](#索引)

### <a id=014>由hdf5資料庫載入待訓練資料</a>

In [ ]:
train_x = pd.read_hdf('ICU_PhysioNet2012.h5', key='train_x')
train_y = pd.read_hdf('ICU_PhysioNet2012.h5', key='train_y')

val_x = pd.read_hdf('ICU_PhysioNet2012.h5', key='val_x')
val_y = pd.read_hdf('ICU_PhysioNet2012.h5', key='val_y')

test_x = pd.read_hdf('ICU_PhysioNet2012.h5', key='test_x')
test_y = pd.read_hdf('ICU_PhysioNet2012.h5', key='test_y')

[[Back to Top]](#索引)

### <a id=015>檢查資料是否有空值</a>

In [ ]:
# 畫出數個病人的資料空值熱圖
# 淺色代表該處為空值，深色代表該處有資料
for idx in pids_train[:3]:
    fig,ax=plt.subplots(figsize=(10,5))
    ax.set_title("Record ID= "+str(idx))
    cmap=sns.light_palette("navy", reverse=False)
    sns.heatmap(train_x.loc[idx].notnull(),yticklabels=False,cmap=cmap,ax=ax)
    plt.show()

In [ ]:
# 檢查張量形狀是否符合預期
assert (train_x.columns == test_x.columns).sum() == train_x.shape[1]
assert (val_x.columns == test_x.columns).sum() == val_x.shape[1]

[[Back to Top]](#索引)

### <a id=016>資料標準化</a>

In [ ]:
cols_to_norm = train_x.columns

for col in cols_to_norm:
    mean = train_x[col].mean()
    std = train_x[col].std()
    train_x[col] = (train_x[col] - mean) / std
    val_x[col] = (val_x[col] - mean) / std
    test_x[col] = (test_x[col] - mean) / std

[[Back to Top]](#索引)

### <a id=017>將空值做填補</a>

In [ ]:
# 看某病患的呼吸率資訊(空值填補前)
pid=val_x.iloc[0].name[0]          #train data內第一個病患的ID
print("patient ID=",pid)
val_x.loc[pid,"RespRate"].plot()
plt.title("Normalized")
plt.ylabel("RespRate")
plt.xlabel("Time")
plt.show()

In [ ]:
# 以forward fill方式填補空值
train_x = train_x.ffill()
test_x = test_x.ffill()
val_x = val_x.ffill()

# 以fillna方式填補剩餘空值
train_x.fillna(value=0, inplace=True)
test_x.fillna(value=0, inplace=True)
val_x.fillna(value=0, inplace=True)

In [ ]:
# 看某病患的呼吸率資訊(空值填補後)
pid=val_x.iloc[0].name[0]          #train data內第一個病患的ID
print("patient ID=",pid)
val_x.loc[pid,"RespRate"].plot()
plt.title("Normalized")
plt.ylabel("RespRate")
plt.xlabel("Time")
plt.show()

[[Back to Top]](#索引)

### <a id=018>將所有病人的時序資料截斷或補零至相同長度</a>

(#samples, #time points, vector length)

In [ ]:
# 略為了解一下，前一萬個病人，他們的時序列長度為何 (x軸為病人ID, y軸為時間點)
train_x.head(10000).groupby(level=0) \
                   .size() \
                   .plot(kind="bar")

以上可見，大多病人生理特徵時序，長度皆位於150以內。

In [ ]:
def data_list_creator(df,pids):
    """於補零或截斷前，須將各病人的時序資料提取出來，放置於清單內。"""
    data_lst=[]
    groups=df.groupby(level=0)
    for pid in pids:
        data_lst.append( groups.get_group(pid).values )
    return data_lst

train_x_list = data_list_creator( train_x, pids_train)
train_y_list = data_list_creator( train_y, pids_train )

test_x_list = data_list_creator( test_x, pids_test )
test_y_list = data_list_creator( test_y, pids_test )

val_x_list = data_list_creator( val_x, pids_val )
val_y_list = data_list_creator( val_y, pids_val )

# 由於Tensor需要固定大小，我們將固定時間序列長度=150 (也就是說，若時序長度未滿150, 則補零，超過，則截斷)
from tensorflow.keras.preprocessing import sequence

train_x_new = sequence.pad_sequences(train_x_list, dtype='float32',
                                     maxlen=150, padding='post',
                                     truncating='post')
test_x_new = sequence.pad_sequences(test_x_list, dtype='float32',
                                    maxlen=150, padding='post', 
                                    truncating='post')
val_x_new = sequence.pad_sequences(val_x_list, dtype='float32',
                                   maxlen=150, padding='post',
                                   truncating='post')

[[Back to Top]](#索引)

## <a id=02>2. 建立模型</a>

In [ ]:
train_x.shape[-1]

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Input, TimeDistributed, Masking
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam

timesteps=150
data_dim=train_x.shape[-1]

# 建立模型
model=Sequential()
# 加入Masking。Masking的用意是：若某時刻資料皆為0,則機器不應去學習該時刻的資料。
# 該時刻的資料應以Masking(遮罩)來處理，使得該時刻的資料不會被納入訓練。
model.add( Masking(mask_value=0., input_shape=(timesteps, data_dim)))
# # 加第一層LSTM
lstm_kwargs = {'dropout': 0.2, 'recurrent_dropout': 0.2, 'return_sequences': True}
model.add( LSTM(256, **lstm_kwargs)
         )
# 加第二層LSTM
lstm_kwargs = {'dropout': 0.2, 'recurrent_dropout': 0.2, 'return_sequences': False}
model.add( LSTM(256, **lstm_kwargs)
         )
# 加入Dense，用以輸出生還機率
model.add( Dense(1, activation='sigmoid')
         )
# 編譯模型
optimizer = Adam()
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=["accuracy"])

# 印出模型摘要
model.summary()

[[Back to Top]](#索引)

## <a id=03>3. 訓練模型</a>

In [ ]:
history = model.fit(train_x_new, train_y.values, batch_size=192, epochs=6,
                    verbose=1,validation_data=[val_x_new, val_y])

# 畫出模型訓練情形。
plt.plot(history.history['acc'],ms=5,marker='o',label='accuracy')
plt.plot(history.history['val_acc'],ms=5,marker='o',label='val accuracy')
plt.legend()
plt.show()

In [ ]:
# 拿建好的模型於test data做出預測
preds = model.predict(test_x_new)

[[Back to Top]](#索引)

## 4. 畫出ROC指標, 驗證模型好壞

https://en.wikipedia.org/wiki/Receiver_operating_characteristic

In [ ]:
from sklearn.metrics import roc_curve, auc

# get 0/1 binary lable for each patient encounter
label = test_y

# get the last prediction in [0,1] for the patient
prediction = preds

# compute ROC curve for predictions
rnn_roc = roc_curve(label,prediction)

# compute the area under the curve of prediction ROC
rnn_auc = auc(rnn_roc[0], rnn_roc[1])

# plot rocs & display AUCs
plt.figure(figsize=(7, 5))
line_kwargs = {'linewidth': 4, 'alpha': 0.8}
plt.plot(rnn_roc[0], rnn_roc[1], label="ROC= %0.3f" % rnn_auc, color='#6AA84F', **line_kwargs)
plt.legend(loc='lower right', fontsize=20)
plt.xlim((-0.05, 1.05))
plt.ylim((-0.05, 1.05))
plt.xticks([0, 0.25, 0.5, 0.75, 1.0], fontsize=14)
plt.yticks([0, 0.25, 0.5, 0.75, 1.0], fontsize=14)
plt.xlabel("False Positive Rate", fontsize=18)
plt.ylabel("True Positive Rate", fontsize=18)
plt.title("Severity of Illness ROC Curves", fontsize=24)
plt.grid(alpha=0.25)
plt.tight_layout()

[[Back to Top]](#索引)

ROC大於0.75, 可見此模型有效。